# Playing with DuckDB

In [1]:
%load_ext lab_black

import duckdb
import pandas as pd
from sklearn.datasets import fetch_openml

In [2]:
# Load data
df = fetch_openml(data_id=42092, as_frame=True)["frame"]

In [3]:
# Initialize duckdb, register df and materialize first query
# If out-of-RAM: duckdb.connect("py.duckdb", config={"temp_directory": "a_directory"})
con = duckdb.connect()
con.register("df", df)
con.execute("SELECT * FROM df limit 5").fetchdf()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,20141013T000000,221900.0,3.0,1.00,1180.0,5650.0,1.0,0.0,0.0,3.0,7.0,1180.0,0.0,1955.0,0.0,98178,47.5112,-122.257,1340.0,5650.0
1,20141209T000000,538000.0,3.0,2.25,2570.0,7242.0,2.0,0.0,0.0,3.0,7.0,2170.0,400.0,1951.0,1991.0,98125,47.7210,-122.319,1690.0,7639.0
2,20150225T000000,180000.0,2.0,1.00,770.0,10000.0,1.0,0.0,0.0,3.0,6.0,770.0,0.0,1933.0,0.0,98028,47.7379,-122.233,2720.0,8062.0
3,20141209T000000,604000.0,4.0,3.00,1960.0,5000.0,1.0,0.0,0.0,5.0,7.0,1050.0,910.0,1965.0,0.0,98136,47.5208,-122.393,1360.0,5000.0
4,20150218T000000,510000.0,3.0,2.00,1680.0,8080.0,1.0,0.0,0.0,3.0,8.0,1680.0,0.0,1987.0,0.0,98074,47.6168,-122.045,1800.0,7503.0


In [4]:
# Average price per grade
query = """
  SELECT AVG(price) avg_price, grade 
  FROM df 
  GROUP BY grade
  ORDER BY grade
  """
avg = con.execute(query).fetchdf()
avg

,avg_price,grade
0,1.420000e+05,1.0
1,2.056667e+05,3.0
2,2.143810e+05,4.0
3,2.485240e+05,5.0
4,3.019196e+05,6.0
5,4.025903e+05,7.0
6,5.428528e+05,8.0
7,7.735132e+05,9.0
8,1.071771e+06,10.0
9,1.496842e+06,11.0


In [5]:
# Save df and avg to different file types
df.to_parquet("housing.parquet")  # pyarrow=7
avg.to_csv("housing_avg.csv", index=False)

In [6]:
# Complex query
query2 = """
  SELECT price, sqft_living, A.grade, avg_price
  FROM 'housing.parquet' A
  LEFT JOIN 'housing_avg.csv' B
  ON A.grade = B.grade
  WHERE B.avg_price > 1000000
  """
expensive_grades = con.execute(query2).fetchdf()
expensive_grades

,price,sqft_living,grade,avg_price
0,1225000.0,5420.0,11.0,1.496842e+06
1,975000.0,2720.0,10.0,1.071771e+06
2,1040000.0,4770.0,11.0,1.496842e+06
3,832500.0,3430.0,10.0,1.071771e+06
4,720000.0,3450.0,10.0,1.071771e+06
...,...,...,...,...
1631,3567000.0,4850.0,10.0,1.071771e+06
1632,1222500.0,4910.0,11.0,1.496842e+06
1633,1088000.0,4170.0,10.0,1.071771e+06
1634,1575000.0,3410.0,10.0,1.071771e+06


In [7]:
con.close()